<a href="https://colab.research.google.com/github/AjeetSingh02/Notebooks/blob/master/pytorchToKerasOnnx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline


(optional) Exporting a Model from PyTorch to ONNX and Running it using ONNX Runtime
========================================================================

In this tutorial, we describe how to convert a model defined
in PyTorch into the ONNX format and then run it with ONNX Runtime.

ONNX Runtime is a performance-focused engine for ONNX models,
which inferences efficiently across multiple platforms and hardware
(Windows, Linux, and Mac and on both CPUs and GPUs).
ONNX Runtime has proved to considerably increase performance over
multiple models as explained `here
<https://cloudblogs.microsoft.com/opensource/2019/05/22/onnx-runtime-machine-learning-inferencing-0-4-release>`__

For this tutorial, you will need to install `ONNX <https://github.com/onnx/onnx>`__
and `ONNX Runtime <https://github.com/microsoft/onnxruntime>`__.
You can get binary builds of ONNX and ONNX Runtime with
``pip install onnx onnxruntime``.
Note that ONNX Runtime is compatible with Python versions 3.5 to 3.7.

``NOTE``: This tutorial needs PyTorch master branch which can be installed by following
the instructions `here <https://github.com/pytorch/pytorch#from-source>`__




In [ ]:
# Some standard imports
import io
import numpy as np

from torch import nn
import torch.utils.model_zoo as model_zoo
import torch.onnx

Super-resolution is a way of increasing the resolution of images, videos
and is widely used in image processing or video editing. For this
tutorial, we will use a small super-resolution model.

First, let's create a SuperResolution model in PyTorch.
This model uses the efficient sub-pixel convolution layer described in
`"Real-Time Single Image and Video Super-Resolution Using an Efficient
Sub-Pixel Convolutional Neural Network" - Shi et al <https://arxiv.org/abs/1609.05158>`__
for increasing the resolution of an image by an upscale factor.
The model expects the Y component of the YCbCr of an image as an input, and
outputs the upscaled Y component in super resolution.

`The
model <https://github.com/pytorch/examples/blob/master/super_resolution/model.py>`__
comes directly from PyTorch's examples without modification:




In [ ]:
# Super Resolution model definition in PyTorch
import torch.nn as nn
import torch.nn.init as init


class SuperResolutionNet(nn.Module):
    def __init__(self, upscale_factor, inplace=False):
        super(SuperResolutionNet, self).__init__()

        self.relu = nn.ReLU(inplace=inplace)
        self.conv1 = nn.Conv2d(1, 64, (5, 5), (1, 1), (2, 2))
        self.conv2 = nn.Conv2d(64, 64, (3, 3), (1, 1), (1, 1))
        self.conv3 = nn.Conv2d(64, 32, (3, 3), (1, 1), (1, 1))
        self.conv4 = nn.Conv2d(32, upscale_factor ** 2, (3, 3), (1, 1), (1, 1))
        self.pixel_shuffle = nn.PixelShuffle(upscale_factor)

        self._initialize_weights()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.relu(self.conv3(x))
        x = self.pixel_shuffle(self.conv4(x))
        return x

    def _initialize_weights(self):
        init.orthogonal_(self.conv1.weight, init.calculate_gain('relu'))
        init.orthogonal_(self.conv2.weight, init.calculate_gain('relu'))
        init.orthogonal_(self.conv3.weight, init.calculate_gain('relu'))
        init.orthogonal_(self.conv4.weight)

# Create the super-resolution model by using the above model definition.
torch_model = SuperResolutionNet(upscale_factor=3)

Ordinarily, you would now train this model; however, for this tutorial,
we will instead download some pre-trained weights. Note that this model
was not trained fully for good accuracy and is used here for
demonstration purposes only.

It is important to call ``torch_model.eval()`` or ``torch_model.train(False)``
before exporting the model, to turn the model to inference mode.
This is required since operators like dropout or batchnorm behave
differently in inference and training mode.




In [ ]:
# Load pretrained model weights
model_url = 'https://s3.amazonaws.com/pytorch/test_data/export/superres_epoch100-44c6958e.pth'
batch_size = 1    # just a random number

# Initialize model with the pretrained weights
map_location = lambda storage, loc: storage
if torch.cuda.is_available():
    map_location = None
torch_model.load_state_dict(model_zoo.load_url(model_url, map_location=map_location))

# set the model to inference mode
torch_model.eval()

Downloading: "https://s3.amazonaws.com/pytorch/test_data/export/superres_epoch100-44c6958e.pth" to /root/.cache/torch/hub/checkpoints/superres_epoch100-44c6958e.pth


SuperResolutionNet(
  (relu): ReLU()
  (conv1): Conv2d(1, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(32, 9, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pixel_shuffle): PixelShuffle(upscale_factor=3)
)

Exporting a model in PyTorch works via tracing or scripting. This
tutorial will use as an example a model exported by tracing.
To export a model, we call the ``torch.onnx.export()`` function.
This will execute the model, recording a trace of what operators
are used to compute the outputs.
Because ``export`` runs the model, we need to provide an input
tensor ``x``. The values in this can be random as long as it is the
right type and size.
Note that the input size will be fixed in the exported ONNX graph for
all the input's dimensions, unless specified as a dynamic axes.
In this example we export the model with an input of batch_size 1,
but then specify the first dimension as dynamic in the ``dynamic_axes``
parameter in ``torch.onnx.export()``.
The exported model will thus accept inputs of size [batch_size, 1, 224, 224]
where batch_size can be variable.

To learn more details about PyTorch's export interface, check out the
`torch.onnx documentation <https://pytorch.org/docs/master/onnx.html>`__.




In [ ]:
# Input to the model
x = torch.randn(batch_size, 1, 224, 224, requires_grad=True)
torch_out = torch_model(x)

# Export the model
torch.onnx.export(torch_model,               # model being run
                  x,                         # model input (or a tuple for multiple inputs)
                  "super_resolution.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable lenght axes
                                'output' : {0 : 'batch_size'}})

We also computed ``torch_out``, the output after of the model,
which we will use to verify that the model we exported computes
the same values when run in ONNX Runtime.

But before verifying the model's output with ONNX Runtime, we will check
the ONNX model with ONNX's API.
First, ``onnx.load("super_resolution.onnx")`` will load the saved model and
will output a onnx.ModelProto structure (a top-level file/container format for bundling a ML model.
For more information `onnx.proto documentation <https://github.com/onnx/onnx/blob/master/onnx/onnx.proto>`__.).
Then, ``onnx.checker.check_model(onnx_model)`` will verify the model's structure
and confirm that the model has a valid schema.
The validity of the ONNX graph is verified by checking the model's
version, the graph's structure, as well as the nodes and their inputs
and outputs.




In [ ]:
! pip install onnx

     |████████████████████████████████| 7.4MB 2.7MB/s 


In [ ]:
import onnx

onnx_model = onnx.load("super_resolution.onnx")
onnx.checker.check_model(onnx_model)

Now let's compute the output using ONNX Runtime's Python APIs.
This part can normally be done in a separate process or on another
machine, but we will continue in the same process so that we can
verify that ONNX Runtime and PyTorch are computing the same value
for the network.

In order to run the model with ONNX Runtime, we need to create an
inference session for the model with the chosen configuration
parameters (here we use the default config).
Once the session is created, we evaluate the model using the run() api.
The output of this call is a list containing the outputs of the model
computed by ONNX Runtime.




In [ ]:
! pip install onnxruntime

     |████████████████████████████████| 3.8MB 2.8MB/s 


In [ ]:
import onnxruntime

ort_session = onnxruntime.InferenceSession("super_resolution.onnx")

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# compute ONNX Runtime output prediction
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(x)}
ort_outs = ort_session.run(None, ort_inputs)

# compare ONNX Runtime and PyTorch results
np.testing.assert_allclose(to_numpy(torch_out), ort_outs[0], rtol=1e-03, atol=1e-05)

print("Exported model has been tested with ONNXRuntime, and the result looks good!")

Exported model has been tested with ONNXRuntime, and the result looks good!


We should see that the output of PyTorch and ONNX Runtime runs match
numerically with the given precision (rtol=1e-03 and atol=1e-05).
As a side-note, if they do not match then there is an issue in the
ONNX exporter, so please contact us in that case.




onnx to keras

In [ ]:
! pip install onnx2keras

  Created wheel for onnx2keras: filename=onnx2keras-0.0.23-cp36-none-any.whl size=24159 sha256=fe1720abe7263b96879776b7c94d6f822d6aaeee48f42b5e3a657df93510e121
  Stored in directory: /root/.cache/pip/wheels/a4/d1/5a/644cd9e957d01977c2e7c5bcb75d8172ba0eab831e2471128d
Successfully built onnx2keras


In [ ]:
from onnx2keras import onnx_to_keras

# Load ONNX model
onnx_model = onnx.load('/content/super_resolution.onnx')

# Call the converter (input - is the main model input name, can be different for your model)
k_model = onnx_to_keras(onnx_model, ['input'])

In [ ]:
k_model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 1, 224, 224)]     0         
_________________________________________________________________
9_pad (ZeroPadding2D)        (None, 1, 228, 228)       0         
_________________________________________________________________
9 (Conv2D)                   (None, 64, 224, 224)      1664      
_________________________________________________________________
10 (Activation)              (None, 64, 224, 224)      0         
_________________________________________________________________
11_pad (ZeroPadding2D)       (None, 64, 226, 226)      0         
_________________________________________________________________
11 (Conv2D)                  (None, 64, 224, 224)      36928     
_________________________________________________________________
12 (Activation)              (None, 64, 224, 224)     

# pytorch model to keras using ONNX

In [ ]:
from torchvision import transforms
# _tasks = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
#     ])

_tasks = transforms.Compose([transforms.ToTensor(),
  transforms.Normalize((0.5,), (0.5,))
])

In [ ]:
from torchvision.datasets import MNIST

In [ ]:
## Load MNIST Dataset and apply transformations
mnist = MNIST("data", download=True, train=True, transform=_tasks)

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw





Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

In [ ]:
## create training and validation split 
split = int(0.8 * len(mnist))
index_list = list(range(len(mnist)))
train_idx, valid_idx = index_list[:split], index_list[split:]

In [ ]:
## create sampler objects using SubsetRandomSampler
tr_sampler = SubsetRandomSampler(train_idx)
val_sampler = SubsetRandomSampler(valid_idx)

In [ ]:
## create iterator objects for train and valid datasets
trainloader = DataLoader(mnist, batch_size=256, sampler=tr_sampler)
validloader = DataLoader(mnist, batch_size=256, sampler=val_sampler)

In [ ]:
import torch.nn.functional as F
import torch.nn as nn
import torch.nn.init as init

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(784, 128)
        self.output = nn.Linear(128, 10)
        
    def forward(self, x):
        x = self.hidden(x)
        x = F.sigmoid(x)
        x = self.output(x)
        return x

In [ ]:
model = Model()

In [ ]:
from torch import optim
import numpy as np
import torch

In [ ]:
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay= 1e-6, momentum = 0.9, nesterov = True)

In [ ]:
for epoch in range(1, 11): ## run the model for 10 epochs
    train_loss, valid_loss = [], []
    ## training part 
    model.train()
    for data, target in trainloader:
        optimizer.zero_grad()
        ## 1. forward propagation
        data = data.view(data.shape[0], -1)
        output = model(data)
        
        ## 2. loss calculation
        loss = loss_function(output, target)
        
        ## 3. backward propagation
        loss.backward()
        
        ## 4. weight optimization
        optimizer.step()
        
        train_loss.append(loss.item())
        
    ## evaluation part 
    model.eval()
    for data, target in validloader:
        data = data.view(data.shape[0], -1)
        output = model(data)
        loss = loss_function(output, target)
        valid_loss.append(loss.item())
    print ("Epoch:", epoch, "Training Loss: ", np.mean(train_loss), "Valid Loss: ", np.mean(valid_loss))

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1625: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch: 1 Training Loss:  1.4182283592985032 Valid Loss:  0.7090614133692802
Epoch: 2 Training Loss:  0.5780729404155244 Valid Loss:  0.4460231883728758
Epoch: 3 Training Loss:  0.4318876773753065 Valid Loss:  0.3720815359277928
Epoch: 4 Training Loss:  0.3753628971728873 Valid Loss:  0.33715897671719813
Epoch: 5 Training Loss:  0.34526101864398795 Valid Loss:  0.3147993658451324
Epoch: 6 Training Loss:  0.32407685836895983 Valid Loss:  0.29944707865410664
Epoch: 7 Training Loss:  0.3084014822193917 Valid Loss:  0.286773940033101
Epoch: 8 Training Loss:  0.29616024955473047 Valid Loss:  0.2755899023502431
Epoch: 9 Training Loss:  0.2849399878623638 Valid Loss:  0.26630250190166715
Epoch: 10 Training Loss:  0.27490580906259254 Valid Loss:  0.26057582554665015


In [ ]:
## dataloader for validation dataset 
dataiter = iter(validloader)
data, labels = dataiter.next()
data = data.view(data.shape[0], -1)
output = model(data)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1625: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [ ]:
_, preds_tensor = torch.max(output, 1)
preds = np.squeeze(preds_tensor.numpy())

In [ ]:
print ("Actual:", labels[:10])
print ("Predicted:", preds[:10])

Actual: tensor([8, 5, 0, 6, 4, 7, 0, 7, 2, 9])
Predicted: [8 5 0 6 4 7 0 7 2 9]


In [ ]:
# Converting to ONNX
x = torch.randn(1, 1, 256, 784, requires_grad=True)

In [ ]:
torch.onnx.export(model, x, "torchToOnnx.onnx", verbose=True, input_names = ['input'], output_names = ['output'])

graph(%input : Float(1:200704, 1:200704, 256:784, 784:1),
      %hidden.bias : Float(128:1),
      %output.bias : Float(10:1),
      %12 : Float(784:1, 128:784),
      %13 : Float(128:1, 10:128)):
  %6 : Float(1:32768, 1:32768, 256:128, 128:1) = onnx::MatMul(%input, %12) # /usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1676:0
  %7 : Float(1:32768, 1:32768, 256:128, 128:1) = onnx::Add(%6, %hidden.bias)
  %8 : Float(1:32768, 1:32768, 256:128, 128:1) = onnx::Sigmoid(%7) # /usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1626:0
  %10 : Float(1:2560, 1:2560, 256:10, 10:1) = onnx::MatMul(%8, %13) # /usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1676:0
  %output : Float(1:2560, 1:2560, 256:10, 10:1) = onnx::Add(%10, %output.bias)
  return (%output)



/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1625: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [ ]:
! pip install onnx2keras

In [ ]:
import onnx
from onnx2keras import onnx_to_keras

In [ ]:
import onnxruntime as rt
import numpy

sess = rt.InferenceSession("/content/torchToOnnx.onnx")
input_name = sess.get_inputs()[0].name
onnxPreds = sess.run(None, {input_name: data.numpy().reshape(1,1,256,784)})[0]

In [ ]:
np.argmax(onnxPreds[0][0][0])

8

In [ ]:
# Load ONNX model
onnx_model = onnx.load('/content/torchToOnnx.onnx')

In [ ]:
# Call the converter (input - is the main model input name, can be different for your model)
k_model = onnx_to_keras(onnx_model, ['input'])

In [ ]:
k_model.summary()

In [ ]:
data[0].shape

torch.Size([784])

In [ ]:
pred = []
for i in range(10):
    inp = data[i].numpy()
    out = k_model.predict(inp.reshape(1, 784))
    pred.append(np.argmax(out))

In [ ]:
preds[:10]

array([4, 4, 7, 2, 9, 8, 1, 5, 8, 7])

In [ ]:
pred[:10]

[4, 4, 7, 2, 9, 8, 1, 5, 8, 7]